#**Generacion de Modelo Personalizado | StyleGAN2-ADA**



In [ ]:
#@title 1. Conectar Colab a  Google Drive
#@markdown Accede a tu Google Drive para cargar tu conjunto de datos, editar las imágenes y guardar los resultados.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title 2. Instalación de StyleGAN2-ADA
#@markdown Ejecuta esta celda para copiar las librerias de StyleGAN2 en tu directorio de Google Drive.
import os, sys, subprocess, importlib, warnings
warnings.filterwarnings('ignore')

# =========================
# 🔧 RUTAS
# =========================
drive_repo_dir = "/content/drive/MyDrive/colab-sg2-ada-pytorch"
repo_subdir = f"{drive_repo_dir}/stylegan2-ada-pytorch"

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["PYTORCH_JIT"] = "0"
os.environ['TORCH_EXTENSIONS_DIR'] = '/tmp/torch_extensions'  # evita escribir en Drive y warnings

# =========================
# 🔍 DETECTAR GPU
# =========================
print("== Detectando GPU ==")
!nvidia-smi -L || echo "⚠️ No se detectó GPU"
import torch
if torch.cuda.is_available():
    print("✅ GPU detectada:", torch.cuda.get_device_name(0))
else:
    print("⚠️ No se detectó GPU en este entorno")

# =========================
# 📦 CLONAR / ACTUALIZAR REPO
# =========================
if not os.path.isdir(repo_subdir):
    print("\n📥 Clonando StyleGAN2-ADA en Drive...")
    os.makedirs(drive_repo_dir, exist_ok=True)
    %cd "$drive_repo_dir"
    !git clone -q https://github.com/NVlabs/stylegan2-ada-pytorch.git
else:
    print("\n🔄 Repositorio existente, actualizando...")
    %cd "$repo_subdir"
    !git fetch origin -q && git reset --hard origin/main -q

%cd "$repo_subdir"
!mkdir -p downloads datasets pretrained

# =========================
# ⚙️ DEPENDENCIAS ESTABLES
# =========================
print("\n🔧 Instalando dependencias conservadoras...")
!pip install -q numpy==1.23.5 scipy==1.10.1 pillow==9.5.0 ninja tqdm opensimplex click requests pyspng imageio-ffmpeg==0.4.3 opencv-python-headless==4.8.1.78

# Colab ya trae torch compatible, aseguramos versión reciente de GPU
import torch
import torchvision
print(f"Torch detectado: {torch.__version__}, GPU: {torch.cuda.is_available()}")

# =========================
# 🧱 EXTENSIONES CUDA (modo silencioso)
# =========================
# Solo intentar recompilar si setup.py existe
torch_ops_path = os.path.join(repo_subdir, "torch_utils", "ops")
if os.path.exists(os.path.join(torch_ops_path, "setup.py")):
    print("\n⚙️ Compilando extensiones CUDA (bias_act / upfirdn2d)...")
    %cd "$torch_ops_path"
    !rm -rf build *.so
    !TORCH_CUDA_ARCH_LIST="8.6;8.0;7.5;7.0" python setup.py build_ext --inplace
    compiled = [f for f in os.listdir(".") if f.endswith(".so")]
    if compiled:
        print(f"✅ Extensiones compiladas: {compiled}")
    else:
        print("⚠️ No se generaron archivos .so — se usará modo lento")
else:
    print("\n⚠️ No se encontró setup.py, se usará modo lento (no hay recompilación)")

%cd "$repo_subdir"

# =========================
# 🔗 AGREGAR AL PATH
# =========================
if repo_subdir not in sys.path:
    sys.path.append(repo_subdir)

# =========================
# ✅ COMPROBACIÓN FINAL
# =========================
import numpy, scipy, PIL, cv2
print("\n📦 Versiones instaladas:")
print(f"  Torch:       {torch.__version__}")
print(f"  Torchvision: {importlib.import_module('torchvision').__version__}")
print(f"  NumPy:       {numpy.__version__}")
print(f"  SciPy:       {scipy.__version__}")
print(f"  Pillow:      {PIL.__version__}")
print(f"  OpenCV:      {cv2.__version__}")

try:
    import dnnlib, legacy
    print("\n✅ StyleGAN2-ADA listo para usar con GPU.")
except Exception as e:
    print(f"⚠️ Error al importar: {e}")

# =========================
# 🔹 OPCIONAL: silenciar upfirdn2d warnings
# =========================
import contextlib, sys
@contextlib.contextmanager
def suppress_stdout_stderr():
    with open(os.devnull, "w") as fnull:
        old_out, old_err = sys.stdout, sys.stderr
        sys.stdout, sys.stderr = fnull, fnull
        try:
            yield
        finally:
            sys.stdout, sys.stderr = old_out, old_err

print("\nℹ️ Instalación completa. Generación de imágenes sin warnings de upfirdn2d se puede lograr usando 'with suppress_stdout_stderr():' alrededor de la carga del modelo.")


In [ ]:
#@title 3. Generación de Imágenes StyleGAN2
import os, sys, warnings, contextlib
import numpy as np
import torch
from PIL import Image
from IPython.display import display, clear_output
import ipywidgets as widgets
import dnnlib, legacy

# --------------------------------------
# Silenciar warnings y salida estándar
# --------------------------------------
warnings.filterwarnings("ignore")
@contextlib.contextmanager
def suppress_output():
    with open(os.devnull, "w") as devnull:
        old_stdout, old_stderr = sys.stdout, sys.stderr
        sys.stdout, sys.stderr = devnull, devnull
        try:
            yield
        finally:
            sys.stdout, sys.stderr = old_stdout, old_stderr

# ===============================
# WIDGETS DE ENTRADA
# ===============================
outdir_widget = widgets.Text(value="/content/drive/MyDrive/Resultados/11", description="Carpeta salida:", layout=widgets.Layout(width='90%'))
network_widget = widgets.Text(value="/content/drive/MyDrive/network-snapshot-000012.pkl", description="Modelo:", layout=widgets.Layout(width='90%'))
trunc_widget = widgets.FloatSlider(value=0.8, min=0.1, max=1.2, step=0.05, description="Trunc ψ:", continuous_update=False, layout=widgets.Layout(width='50%'))
seeds_widget = widgets.Text(value="1200-1202", description="Seeds:", layout=widgets.Layout(width='60%'))
gen_random_widget = widgets.Checkbox(value=False, description="Generar aleatorias")
n_imgs_widget = widgets.IntSlider(value=3, min=1, max=100, step=1, description="Cantidad:", continuous_update=False, layout=widgets.Layout(width='50%'))

def toggle_n_imgs(change):
    n_imgs_widget.disabled = not change['new']
gen_random_widget.observe(toggle_n_imgs, names='value')
toggle_n_imgs({'new': gen_random_widget.value})

gen_nonsquare_widget = widgets.Checkbox(value=False, description="Generar no cuadradas")
width_widget = widgets.IntText(value=640, description="Ancho:")
height_widget = widgets.IntText(value=480, description="Alto:")

# ===============================
# Funciones auxiliares
# ===============================
def pad_and_resize(img, target_width, target_height):
    w, h = img.size
    scale = min(target_width / w, target_height / h)
    new_w, new_h = int(w * scale), int(h * scale)
    img_resized = img.resize((new_w, new_h), Image.LANCZOS)
    canvas = Image.new("RGB", (target_width, target_height))
    pad_left, pad_top = (target_width - new_w)//2, (target_height - new_h)//2
    canvas.paste(img_resized, (pad_left, pad_top))
    return canvas

# ===============================
# Función principal
# ===============================
def generate_images(b):
    clear_output(wait=True)
    display(ui)
    with output_area:
        clear_output(wait=True)
        print("🔄 Generando imágenes...\n")

        outdir = outdir_widget.value.strip()
        network_path = network_widget.value.strip()
        truncation = trunc_widget.value
        gen_random = gen_random_widget.value
        n_imgs = n_imgs_widget.value
        seeds_input = seeds_widget.value.strip()
        gen_nonsquare = gen_nonsquare_widget.value
        width, height = width_widget.value, height_widget.value

        os.makedirs(outdir, exist_ok=True)
        device = "cuda" if torch.cuda.is_available() else "cpu"

        # Cargar modelo silenciosamente
        print("🧠 Cargando modelo...")
        with suppress_output():
            with dnnlib.util.open_url(network_path) as f:
                G = legacy.load_network_pkl(f)['G_ema'].to(device)
        print("✅ Modelo cargado.\n")

        # Preparar seeds
        if gen_random:
            seeds_list = list(np.random.randint(0, 4294967295, size=n_imgs))
        else:
            seeds_str = seeds_input.replace(" ", "")
            if '-' in seeds_str:
                start, end = [int(x) for x in seeds_str.split('-')]
                seeds_list = list(range(start, end+1))
            else:
                seeds_list = [int(s) for s in seeds_str.split(',')]

        print(f"🔹 Generando {len(seeds_list)} imágenes...")

        for seed in seeds_list:
            z = np.random.RandomState(seed).randn(G.z_dim)  # Shape (512,)
            z_tensor = torch.from_numpy(z).to(device).unsqueeze(0)
            with suppress_output():
                img_tensor = G(z_tensor, None, truncation_psi=truncation, noise_mode='const')
            img = ((img_tensor.clamp(-1,1)+1)*127.5).permute(0,2,3,1).cpu().numpy().astype(np.uint8)
            pil_img = Image.fromarray(img[0])

            if gen_nonsquare:
                pil_img = pad_and_resize(pil_img, width, height)

            filename = os.path.join(outdir, f"img_{seed}.png")
            pil_img.save(filename)
            print(f"✅ Guardada: {filename}")

        print(f"\n🎉 Generación completada sin errores ni mensajes molestos. ({len(seeds_list)} imágenes)")

# ===============================
# Interfaz final
# ===============================
section_trunc_seeds = widgets.VBox([trunc_widget, seeds_widget])
section_random = widgets.VBox([gen_random_widget, n_imgs_widget])
section_nonsquare = widgets.VBox([gen_nonsquare_widget, width_widget, height_widget])

generate_button = widgets.Button(description="🚀 Generar Imágenes", button_style='success')
output_area = widgets.Output()

ui = widgets.VBox([
    outdir_widget,
    network_widget,
    section_trunc_seeds,
    section_random,
    section_nonsquare,
    generate_button,
    output_area
])

generate_button.on_click(generate_images)
display(ui)


In [ ]:
#@title 4. Explorador Avanzado de Espacio Latente
import torch, sys, os, contextlib, warnings, numpy as np
from PIL import Image
from IPython.display import display, clear_output
import ipywidgets as widgets
import dnnlib, legacy

# ---------------------------
# 🔹 Silenciar warnings y salida molesta
# ---------------------------
warnings.filterwarnings("ignore")

@contextlib.contextmanager
def suppress_output():
    """Silencia stdout y stderr temporalmente."""
    with open(os.devnull, "w") as devnull:
        old_stdout, old_stderr = sys.stdout, sys.stderr
        sys.stdout, sys.stderr = devnull, devnull
        try:
            yield
        finally:
            sys.stdout, sys.stderr = old_stdout, old_stderr

# ---------------------------
# 🔹 Configuración inicial
# ---------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚡ Usando device: {device}")

latent_dim = 512
num_sliders = 5
min_image_size = 128
max_image_size = 1024

# ---------------------------
# 🔹 Widgets principales
# ---------------------------
network_widget = widgets.Text(
    value="/content/drive/MyDrive/network-snapshot-000012.pkl",
    description="Modelo:",
    layout=widgets.Layout(width='90%')
)

output_dir_widget = widgets.Text(
    value="/content/drive/MyDrive/resultados_latente",
    description="Carpeta salida:",
    layout=widgets.Layout(width='90%')
)

load_button = widgets.Button(description="🧠 Cargar Modelo", button_style='info')

mode_widget = widgets.Dropdown(
    options=["Seed base", "Cargar snapshot desde ruta"],
    value="Seed base",
    description="Modo:",
    layout=widgets.Layout(width='50%')
)

seed_widget = widgets.IntText(
    value=500,
    description="Seed base:",
    layout=widgets.Layout(width='50%')
)

snapshot_path_widget = widgets.Text(
    value="/content/drive/MyDrive/resultados_latente/z_snapshot_103.npy",
    description="Snapshot path:",
    layout=widgets.Layout(width='90%')
)

init_button = widgets.Button(description="⚡ Inicializar Vector Base", button_style='info')

trunc_widget = widgets.FloatSlider(
    value=0.5, min=0.1, max=1.2, step=0.01,
    description="Trunc ψ:", continuous_update=True,
    layout=widgets.Layout(width='50%')
)

# --- Explorador latente ---
sliders = {}
for i in range(num_sliders):
    sliders[str(i)] = widgets.FloatSlider(
        value=0.0, min=-5.0, max=5.0, step=0.1,
        description=f'Slider {i+1}', continuous_update=True,
        layout=widgets.Layout(width='90%')
    )

image_size_slider = widgets.IntSlider(
    value=512, min=min_image_size, max=max_image_size,
    step=64, description='Tamaño imagen:',
    continuous_update=True, layout=widgets.Layout(width='90%')
)

slider_box = widgets.VBox(list(sliders.values()) + [image_size_slider])

# --- Guardar snapshot ---
save_index_slider = widgets.IntSlider(
    value=0, min=0, max=999, step=1,
    description='Snapshot #:', layout=widgets.Layout(width='90%')
)
save_button = widgets.Button(description="💾 Guardar snapshot", button_style='success')

output_area = widgets.Output()

# ---------------------------
# 🔹 Funciones principales
# ---------------------------
def load_model(b=None):
    network_pkl = network_widget.value.strip()
    print("🔄 Cargando modelo...")
    global G
    with suppress_output():
        with dnnlib.util.open_url(network_pkl) as f:
            G = legacy.load_network_pkl(f)['G_ema'].to(device)
    print("✅ Modelo cargado sin mensajes molestos.")

def init_latent(b=None):
    global z_base
    mode = mode_widget.value
    seed = seed_widget.value
    snapshot_path = snapshot_path_widget.value.strip()

    if mode == "Cargar snapshot desde ruta" and os.path.isfile(snapshot_path):
        print(f"📂 Cargando snapshot desde: {snapshot_path}")
        z_base = np.load(snapshot_path)
    else:
        print(f"🎲 Usando seed base: {seed}")
        rng = np.random.RandomState(seed)
        z_base = rng.randn(latent_dim)

    update_image()

def generate_image(z, img_size):
    z_tensor = torch.from_numpy(z).to(device).unsqueeze(0)
    with suppress_output():
        img = G(z_tensor, None, truncation_psi=trunc_widget.value, noise_mode='const')
    img = (img.clamp(-1,1)+1)*(255/2)
    img = img.permute(0,2,3,1).cpu().numpy().astype(np.uint8)
    pil_img = Image.fromarray(img[0]).resize((img_size,img_size), Image.LANCZOS)
    return pil_img

slider_matrix = np.random.randn(num_sliders, latent_dim) * 0.5

def update_image(change=None):
    z = z_base.copy()
    for i in range(num_sliders):
        z += sliders[str(i)].value * slider_matrix[i]
    img = generate_image(z, image_size_slider.value)
    with output_area:
        clear_output(wait=True)
        display(img)

def save_snapshot(z, img, index):
    output_dir = output_dir_widget.value.strip()
    os.makedirs(output_dir, exist_ok=True)
    np.save(os.path.join(output_dir, f"z_snapshot_{index:03d}.npy"), z)
    img.save(os.path.join(output_dir, f"img_snapshot_{index:03d}.png"))
    print(f"💾 Snapshot guardado correctamente: {index:03d}")

def on_save_clicked(b):
    z = z_base.copy()
    for i in range(num_sliders):
        z += sliders[str(i)].value * slider_matrix[i]
    img = generate_image(z, image_size_slider.value)
    save_snapshot(z, img, save_index_slider.value)

# ---------------------------
# 🔹 Conectar eventos
# ---------------------------
load_button.on_click(load_model)
init_button.on_click(init_latent)
save_button.on_click(on_save_clicked)

for s in sliders.values():
    s.observe(update_image, names='value')
image_size_slider.observe(update_image, names='value')

# ---------------------------
# 🔹 Ocultar/mostrar según modo
# ---------------------------
def toggle_mode(change):
    if change['new'] == "Seed base":
        seed_widget.layout.display = 'block'
        snapshot_path_widget.layout.display = 'none'
    else:
        seed_widget.layout.display = 'none'
        snapshot_path_widget.layout.display = 'block'

mode_widget.observe(toggle_mode, names='value')
toggle_mode({'new': mode_widget.value})

# ---------------------------
# 🔹 Interfaz final
# ---------------------------
config_box = widgets.VBox([network_widget, output_dir_widget, load_button])
vector_box = widgets.VBox([mode_widget, seed_widget, snapshot_path_widget, init_button, trunc_widget])
explorer_box = widgets.VBox([slider_box])
save_box = widgets.VBox([save_index_slider, save_button])

ui = widgets.VBox([config_box, vector_box, explorer_box, save_box, output_area])
display(ui)


In [ ]:
#@title 5. Interpolador StyleGAN2 → Video
#@markdown Permite seleccionar Seeds o Snapshots como entrada para "interpolar" y genera un video en formato mp4.
import os, sys, contextlib
import numpy as np
import torch
from PIL import Image
import imageio.v2 as imageio
import dnnlib, legacy
import warnings

# ========================================
# ⚠️ Supresión de stdout/stderr y warnings
# ========================================
@contextlib.contextmanager
def suppress_all():
    with open(os.devnull,"w") as fnull:
        old_stdout, old_stderr = sys.stdout, sys.stderr
        old_warn = warnings.showwarning
        sys.stdout, sys.stderr = fnull, fnull
        warnings.showwarning = lambda *a,**k: None
        try:
            yield
        finally:
            sys.stdout, sys.stderr = old_stdout, old_stderr
            warnings.showwarning = old_warn

# ========================================
# #param Entradas (interfaz en Colab)
# ========================================
# Tipo de interpolación
interp_type = "Seeds" #@param ["Seeds","Snapshots"]
# Modelo
network_pkl = "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00014-verifiicado-paper512-noaug-resumecustom/network-snapshot-000000.pkl" #@param {type:"string"}
# Carpeta de salida
output_dir = "/content/drive/MyDrive/Test_StyleGAN/resultados/1" #@param {type:"string"}
video_name = "interpolacion.mp4" #@param {type:"string"}
fps = 30 #@param {type:"integer"}
duracion = 5 #@param {type:"number"}
trunc = 0.5 #@param {type:"number"}
aspect_ratio = "1:1" #@param ["1:1","4:5","5:4","16:9","9:16"]
loop_video = True #@param {type:"boolean"}

# ---------- Opciones Seeds ----------
seeds_input = "1200-1250" #@param {type:"string"}
gen_random = True #@param {type:"boolean"}
n_imgs = 3 #@param {type:"integer"}

# ---------- Opciones Snapshots ----------
snapshots_dir = "/content/drive/MyDrive/resultados_latente" #@param {type:"string"}
snapshots_input = "000,001" #@param {type:"string"}

# ========================================
# Funciones auxiliares
# ========================================
def pad_symm(img,target_width,target_height):
    w,h = img.size
    scale = min(target_width/w,target_height/h)
    new_w,new_h = max(int(round(w*scale)),1), max(int(round(h*scale)),1)
    img_resized = img.resize((new_w,new_h), Image.LANCZOS)
    canvas = Image.new("RGB",(target_width,target_height))
    pad_left,pad_top = (target_width-new_w)//2,(target_height-new_h)//2
    canvas.paste(img_resized,(pad_left,pad_top))
    return canvas

def apply_aspect_ratio(img,target_ratio):
    w,h = img.size
    ratio_map = {'1:1':1,'4:5':4/5,'5:4':5/4,'16:9':16/9,'9:16':9/16}
    ar = ratio_map[target_ratio]
    if w/h>ar:
        new_w,new_h = w,int(round(w/ar))
    else:
        new_h,new_w = h,int(round(h*ar))
    return pad_symm(img,new_w,new_h)

# ========================================
# Función principal de interpolación
# ========================================
def generate_interpolation():
    os.makedirs(output_dir,exist_ok=True)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    print("🧠 Cargando modelo...")
    with suppress_all():
        with dnnlib.util.open_url(network_pkl) as f:
            G = legacy.load_network_pkl(f)['G_ema'].to(device)
    print("✅ Modelo cargado.\n")

    latent_vectors = []

    # ---------- Lógica según tipo de interpolación ----------
    if interp_type=="Snapshots":
        print("💾 Modo Snapshots activado")
        indices = [s.strip() for s in snapshots_input.split(",") if s.strip()]
        for idx in indices:
            path = os.path.join(snapshots_dir,f"z_snapshot_{idx}.npy")
            if os.path.isfile(path):
                latent_vectors.append(np.load(path))
                print(f"📂 Snapshot cargado: {os.path.basename(path)}")
            else:
                print(f"⚠️ No se encontró: {path}")
        if len(latent_vectors)<2:
            print("❌ Se necesitan al menos 2 snapshots."); return
    else:
        print("🎲 Modo Seeds activado")
        if gen_random:
            seeds_list = list(np.random.randint(0,4294967295,size=n_imgs))
        else:
            if '-' in seeds_input:
                start,end = map(int,seeds_input.split('-'))
                seeds_list = list(range(start,end+1))
            else:
                seeds_list = [int(s) for s in seeds_input.split(',')]
        print(f"🔹 Seeds a generar: {seeds_list}")
        for seed in seeds_list:
            z = np.random.RandomState(seed).randn(G.z_dim)
            latent_vectors.append(z)

    if loop_video:
        latent_vectors.append(latent_vectors[0])

    # ========================================
    # Generación de frames
    # ========================================
    img_res = G.img_resolution
    def generate_image(z):
        z_tensor = torch.from_numpy(z).to(device).unsqueeze(0)
        with torch.no_grad(), suppress_all():
            img_tensor = G(z_tensor,None,truncation_psi=trunc,noise_mode='const')
        img = ((img_tensor.clamp(-1,1)+1)*127.5).permute(0,2,3,1).cpu().numpy().astype(np.uint8)
        pil_img = Image.fromarray(img[0]).resize((img_res,img_res),Image.LANCZOS)
        pil_img = apply_aspect_ratio(pil_img,aspect_ratio)
        return pil_img

    num_segments = len(latent_vectors)-1
    frames_total = int(fps*duracion)
    frames_per_segment = frames_total//num_segments
    print(f"\n🎞️ Generando {frames_total} frames ({frames_per_segment} por segmento)...")

    frames=[]
    for i in range(num_segments):
        z_start,z_end = latent_vectors[i],latent_vectors[i+1]
        for t in np.linspace(0,1,frames_per_segment,endpoint=False):
            z_interp = (1-t)*z_start + t*z_end
            frames.append(np.array(generate_image(z_interp)))
            print(f"\r🌀 Frame {len(frames)}/{frames_total}",end="")
    frames.append(np.array(generate_image(latent_vectors[-1])))

    video_path = os.path.join(output_dir,video_name)
    imageio.mimsave(video_path,frames,fps=fps,quality=8)
    print(f"\n✅ Video generado: {video_path}")

# ========================================
# Ejecutar generación
# ========================================
generate_interpolation()
